<a href="https://colab.research.google.com/github/ThatiFlowers/Treinamento/blob/main/CRUD_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import sqlite3

def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

def create_tables():
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        preco REAL NOT NULL,
        categoria TEXT NOT NULL
    )
    """)

    conexao.commit()
    conexao.close()

create_tables()


In [38]:
from os import name
# Criando a classe de produtos/ repositório(domínio)
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        return f"Produto(id={self.id}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

In [39]:
# Crie uma pasta 'repository'no ambiente notebook
!mkdir -p repository

In [40]:
%%writefile repository/produto_repository.py
import sqlite3
from typing import List, Optional
from models.produto import Produto

Overwriting repository/produto_repository.py


In [41]:
def cadastrar_produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )

    conexao.commit()
    conexao.close()


In [42]:
def limpar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("DELETE FROM produtos")
    conexao.commit()
    conexao.close()



In [43]:
#teste

produto1 = produto("cocavinho",12, "bebida")
produto2 = produto("bolorosa",12.59,"comida")
produto3 = produto("brownie", 11, "comida")
cadastrar_produto(produto1)
cadastrar_produto(produto2)
cadastrar_produto(produto3)

In [44]:
def listar_produtos():
  conexao = conexao_bd()
  cursor = conexao.cursor()
  cursor.execute("select * from produtos")
  rows = cursor.fetchall()
  conexao.close()
  return rows

print("Produtos cadastrados:")
for produto_data in listar_produtos():
    # Assuming the order is id, nome, preco, categoria based on your create_tables()
    print(f"ID: {produto_data[0]}, Nome: {produto_data[1]}, Preço: {produto_data[2]}, Categoria: {produto_data[3]}")

Produtos cadastrados:
ID: 1, Nome: cocavinho, Preço: 12.0, Categoria: bebida
ID: 2, Nome: bolorosa, Preço: 12.59, Categoria: comida
ID: 3, Nome: brownie, Preço: 11.0, Categoria: comida
ID: 4, Nome: cocavinho, Preço: 12.0, Categoria: bebida
ID: 5, Nome: bolorosa, Preço: 12.59, Categoria: comida
ID: 6, Nome: brownie, Preço: 11.0, Categoria: comida
ID: 7, Nome: cocavinho, Preço: 12.0, Categoria: bebida
ID: 8, Nome: bolorosa, Preço: 12.59, Categoria: comida
ID: 9, Nome: brownie, Preço: 11.0, Categoria: comida


In [45]:
# Original definition of listar_produtos moved to cell 15ee9bbd

In [46]:
for p in listar_produtos():
    print(p[1], p[2], p[3])

cocavinho 12.0 bebida
bolorosa 12.59 comida
brownie 11.0 comida
cocavinho 12.0 bebida
bolorosa 12.59 comida
brownie 11.0 comida
cocavinho 12.0 bebida
bolorosa 12.59 comida
brownie 11.0 comida


In [47]:
# covertendo em uma lista de produtos
def converter_rows_para_produtos(rows):
    lista_de_produtos = []
    for row in rows:
        # formato: (id, nome, preco, categoria)
        p = produto(row[1], row[2], row[3])
        lista_de_produtos.append(p)
    return lista_de_produtos

def produtos_id(id):
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute("select * from produtos where id =?", (id,))
    row = cursor.fetchall()
    conexao.close()
    if row:
        p = produto(row[1], row[2], row[3])
        return p
    return None # Retorna "none" se o produto for encontrado com ID conhecido

In [53]:
from models.product_model import ProductModel

def get(self) -> list:
    try:
        cursor = self.conn.cursor()
        cursor.execute("SELECT id, name, price, stock FROM products")

        rows = cursor.fetchall()
        ret = []

        for row in rows:
            product = ProductModel(
                id=row[0],
                name=row[1],
                price=row[2],
                stock=row[3]
            )
            ret.append(product)

        logging.debug(f"[PRODUCT-STORAGE] Retrieved {len(ret)} products")
        return ret
    except Exception as error:
        logging.error(f"[PRODUCT-STORAGE] Fail to get products: {error}")
        raise error

In [52]:
from models.product_model import ProductModel

def insert(self, product: ProductModel) -> int:
    try:
        cursor = self.conn.cursor()
        cursor.execute("""
            INSERT INTO products (name, price, stock)
            VALUES (?, ?, ?)
        """, (product.name, product.price, product.stock))

        self.conn.commit()
        product_id = cursor.lastrowid

        logging.debug(f"[PRODUCT-STORAGE] Product inserted with ID {product_id}")
        return product_id
    except Exception as error:
        logging.error(f"[PRODUCT-STORAGE] Fail to insert product: {error}")
        raise error

In [50]:
# Crie uma pasta 'models'no ambiente notebook
!mkdir -p models

In [51]:
%%writefile models/product_model.py
from typing import Optional

class ProductModel:
    def __init__(self, name: str, price: float, stock: int, id: Optional[int] = None):
        self.id = id
        self.name = name
        self.price = price
        self.stock = stock

    def __repr__(self):
        return f"ProductModel(id={self.id}, name='{self.name}', price={self.price}, stock={self.stock})"

Writing models/product_model.py


# Task
Install Flask, create a Flask application with GET and POST endpoints for products, utilizing existing `listar_produtos` and `cadastrar_produto` functions, and run the application.

## Instalar Flask

### Subtask:
Instalar a biblioteca Flask no ambiente para habilitar a criação da API web.


**Reasoning**:
The subtask is to install the Flask library. This can be achieved by running the `pip install Flask` command in a code cell.



In [54]:
pip install Flask

## Criar Aplicação Flask

### Subtask:
Configurar o esqueleto de uma aplicação Flask, incluindo a importação das bibliotecas necessárias e a inicialização do aplicativo.


**Reasoning**:
To configure the Flask application skeleton and make the existing database functions and the `Produto` class available, I will create a new file named `app.py` using `%%writefile`. This file will include the necessary Flask imports and initialization, along with the definitions of `conexao_bd`, `Produto`, `cadastrar_produto`, and `listar_produtos` to ensure they are available within the Flask application's scope as per the instruction to 'import from your environment'.



In [55]:
%%writefile app.py
from flask import Flask
import sqlite3

# Database connection function (from WEFJx95LjIDo)
def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

# Produto class definition (from TaQ1X8qxkCjR)
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        # Note: The original __repr__ assumes an 'id' attribute, which is not set during __init__.
        # This might cause an AttributeError if an ID is not explicitly assigned to the object.
        return f"Produto(id={getattr(self, 'id', 'N/A')}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

# Function to register a product (from 5BT5e7cBm0Iv)
def cadastrar_produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )
    conexao.commit()
    conexao.close()

# Function to list products (from 15ee9bbd)
def listar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("select * from produtos")
    rows = cursor.fetchall()
    conexao.close()
    return rows

# Initialize Flask application
app = Flask(__name__)

# Further Flask routes will be added here

Writing app.py


## Definir Endpoint GET /produtos

### Subtask:
Criar uma rota GET para '/produtos' que retorne a lista de todos os produtos cadastrados no banco de dados, utilizando a função `listar_produtos` e convertendo os resultados para o formato JSON.


**Reasoning**:
To implement the GET endpoint for '/produtos', I need to modify the `app.py` file to include the necessary Flask imports (`request`, `jsonify`), define the new route, call the `listar_produtos` function, convert the results into a list of dictionaries, and then return them as JSON.



In [56]:
%%writefile app.py
from flask import Flask, request, jsonify
import sqlite3

# Database connection function
def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

# Produto class definition
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        return f"Produto(id={getattr(self, 'id', 'N/A')}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

# Function to register a product
def cadastrar_produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )
    conexao.commit()
    conexao.close()

# Function to list products
def listar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("select * from produtos")
    rows = cursor.fetchall()
    conexao.close()
    return rows

# Initialize Flask application
app = Flask(__name__)

# Define GET endpoint for /produtos
@app.route('/produtos', methods=['GET'])
def get_produtos():
    produtos_db = listar_produtos()
    produtos_json = []
    for p_data in produtos_db:
        produtos_json.append({
            'id': p_data[0],
            'nome': p_data[1],
            'preco': p_data[2],
            'categoria': p_data[3]
        })
    return jsonify(produtos_json)


Overwriting app.py


## Definir Endpoint POST /produtos

### Subtask:
Criar uma rota POST para '/produtos' que permita adicionar novos produtos ao banco de dados. Este endpoint receberá os dados do produto via JSON na requisição e utilizará a função `cadastrar_produto`.


**Reasoning**:
To implement the POST endpoint for '/produtos', I need to modify the `app.py` file to include the 'POST' method in the route decorator, and then add conditional logic within the `get_produtos` function to handle incoming POST requests by parsing JSON data, creating a `Produto` object, calling `cadastrar_produto`, and returning a success message with a 201 status.



In [57]:
%%writefile app.py
from flask import Flask, request, jsonify
import sqlite3

# Database connection function
def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

# Produto class definition
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        return f"Produto(id={getattr(self, 'id', 'N/A')}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

# Function to register a product
def cadastrar_produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )
    conexao.commit()
    conexao.close()

# Function to list products
def listar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("select * from produtos")
    rows = cursor.fetchall()
    conexao.close()
    return rows

# Initialize Flask application
app = Flask(__name__)

# Define GET and POST endpoint for /produtos
@app.route('/produtos', methods=['GET', 'POST'])
def handle_produtos():
    if request.method == 'GET':
        produtos_db = listar_produtos()
        produtos_json = []
        for p_data in produtos_db:
            produtos_json.append({
                'id': p_data[0],
                'nome': p_data[1],
                'preco': p_data[2],
                'categoria': p_data[3]
            })
        return jsonify(produtos_json)

    elif request.method == 'POST':
        data = request.get_json()
        if not data or not all(k in data for k in ('nome', 'preco', 'categoria')):
            return jsonify({'error': 'Dados incompletos para o produto'}), 400

        nome = data['nome']
        preco = data['preco']
        categoria = data['categoria']

        try:
            novo_produto = Produto(nome, preco, categoria)
            cadastrar_produto(novo_produto)
            return jsonify({'message': 'Produto cadastrado com sucesso', 'produto': {'nome': nome, 'preco': preco, 'categoria': categoria}}), 201
        except Exception as e:
            return jsonify({'error': str(e)}), 500


Overwriting app.py


## Executar Aplicação Flask

### Subtask:
Adicionar o código para executar a aplicação Flask, tornando os endpoints acessíveis. É importante notar que em um ambiente de notebook, o acesso externo pode exigir ferramentas adicionais como ngrok, mas a execução local será demonstrada.


**Reasoning**:
To execute the Flask application, I will create a new code cell that imports the `app` object from `app.py` and then runs it using `app.run()` with a specified host and port to make it accessible within the Colab environment. This will start the web server.



In [ ]:
from app import app

if __name__ == '__main__':
    # Run the Flask app on host='0.0.0.0' to make it accessible externally
    # within the Colab environment. The default port is 5000.
    # For external access outside Colab, consider using ngrok or similar tools.
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
